In [4]:
S = "banana"
S = S + "$"

In [23]:
# Step 1: Comput N_1[i].

import collections

def compute_n1(S):
    letter_count = collections.defaultdict(int)
    sorted_letters = sorted(set([letter for letter in S]))

    for letter in S:
        letter_count[letter] += 1

    letters_before = dict()
    running_total = 0
    for l in sorted_letters:
        letters_before[l] = running_total
        running_total += letter_count[l]

    n = len(S)
    N_1 = [0 for i in range(n)]
    for i in range(n):
        l = S[i]
        N_1[i] = letters_before[l]
    
    return N_1





In [16]:
# Implement ternary quicksort. Will be used in Step 2 below.
import random

def ternary_quicksort(A):    
    n = len(A)
    if n <= 4:
        return sorted(A)
    
    i = random.randint(0, n - 1)
    split_element = A[i]
    A_less = []
    A_eq = []
    A_greater = []
    for a in A:
        if a < split_element:
            A_less.append(a)
        elif a == split_element:
            A_eq.append(a)
        else:
            A_greater.append(a)
    A_sorted = ternary_quicksort(A_less) + A_eq + ternary_quicksort(A_greater)
    return A_sorted
    

In [ ]:
ternary_quicksort([9, 2, 15, 4, 3, 2, 2, 3, 2, 3, 12, 11, 5, 1, 5, 5, 7, 8, 9, 6, 10, 14, 13, 3, 9, 9, 9, 4])

In [21]:
# Step 2: From N_l, compute N_2l.

# These will be our parameters when we make this a function.



def compute_n2l(N_l, l):
    n = len(N_l)
    def N_i_plus_l(i, l):
        if i + l >= n:
            return n
        else:
            return N_l[i + l]

    N_l_pairs = []
    index_to_pair = dict()
    for i in range(n):
        pair = (N_l[i], N_i_plus_l(i, l))
        index_to_pair[i] = pair
        N_l_pairs.append(pair)

    N_l_pairs_sorted = ternary_quicksort(N_l_pairs)


    pairs_before = dict()
    prev = None
    count = 0
    for i in range(n):
        pair = N_l_pairs_sorted[i]
        if pair == prev:
            count += 1
        else:
            pairs_before[pair] = count
            count += 1
            prev = pair

    N_2l = [0 for i in range(n)]
    for i in range(n):
        pair = index_to_pair[i]
        N_2l[i] = pairs_before[pair]
    return N_2l










N_2l

In [26]:
import math

# We assume that S has already been padded with '$' at the end.
def compute_inverse_suffix_array(S):
    n = len(S)
    l = 1
    N_l = compute_n1(S)
    k = math.ceil(math.log2(n))
    l_max = 2 ** k
    while l <= l_max:
        N_2l = compute_n2l(N_l, l)
        l = l * 2
        N_l = N_2l
    return N_l

        

    

In [52]:
# We assume '$' has been appended to the end of S
def compute_suffix_array(S):
    n = len(S)
    SA_inverse = compute_inverse_suffix_array(S)
    SA = [-1] * n
    for i in range(n):
        SA[SA_inverse[i]] = i
    return SA, SA_inverse



In [28]:
S = 'banana$'
SA_inverse = compute_inverse_suffix_array(S)
print(SA_inverse)

n = len(S)
SA = [-1] * n
for i in range(n):
    SA[SA_inverse[i]] = i

print(SA)


[4, 3, 6, 2, 5, 1, 0]
[6, 5, 3, 1, 0, 4, 2]


In [45]:
S = 'bananasareyummy$'
SA, SA_inverse = compute_suffix_array(S)
n = len(S) + 1
print("n: ", n)
print(SA)
print(SA_inverse)

n:  16
[15, 1, 3, 7, 5, 0, 9, 12, 13, 2, 4, 8, 6, 11, 14, 10]
[5, 1, 9, 2, 10, 4, 12, 3, 11, 6, 15, 13, 7, 8, 14, 0]


In [53]:
S = 'bananas$'
n = len(S)
SA, SA_inverse = compute_suffix_array(S)
print("SA:    ", SA)
print("SA -1: ", SA_inverse)

SA:     [7, 1, 3, 5, 0, 2, 4, 6]
SA -1:  [4, 1, 5, 2, 6, 3, 7, 0]


In [59]:

def compute_longest_common_prefixes(SA, SA_inverse, T):
    n = len(T)
    # LCP[i] = longest common prefix between SA[i] and SA[i - 1]
    LCP = [-1] * n

    # We know SA_inverse[n - 1] = 0 (the last character is the special '$')
    # We don't want to compute LCP[0], so we go to i = n - 2
    l = 0
    for i in range(n - 1):
        k = SA_inverse[i]  # What order is the prefix Ti
        j = SA[k - 1]      # Prefix Tj comes immediately before Ti in lexicographic order
        print("k ", k, "j ", j)
        while T[i + l] == T[j + l]:
            l = l + 1
        LCP[k] = l
        if l > 0:
            l = l - 1
    return LCP

